### Environment.state and Constraints

Last updated: 01/22/2020

In [1]:
import numpy as np
import pandas as pd

from marmot import Environment, Agent, process, lt

#### State Data

In [2]:
# The marmot.Environment class can also be configured with time series that represent the
# state of the environment throughout time. This dataset can then be used to dynamically
# constrain tasks as they occur throughout the simulation.


# Load data
data = pd.read_csv("test_data.csv")\
         .set_index("datetime")\
         .to_records()  # The input data type should be np.ndarray

# To create an Environment with state data:
env = Environment("Test Environment", state=data)

# The current state forecast can be accessed through env.state:
env.state

rec.array([( 1, 12, 1,  True), ( 2,  6, 1,  True), ( 3,  7, 1, False),
           ( 4,  4, 1, False), ( 5,  3, 1, False), ( 6, 12, 2, False),
           ( 7, 12, 2, False), ( 8, 12, 2, False), ( 9,  5, 2, False),
           (10,  6, 2, False), (11,  7, 2, False), (12,  3, 2, False),
           (13,  2, 3, False), (14,  3, 3, False), (15,  4, 3, False),
           (16,  5, 3, False), (17,  1, 2, False), (18,  2, 2, False),
           (19,  5, 2, False), (20,  8, 2, False), (21,  9, 2, False),
           (22,  8, 2, False), (23,  5, 3, False), (24,  4, 3, False),
           (25,  3, 3, False), (26,  2, 3,  True), (27,  1, 3,  True),
           (28,  8, 3,  True), (29,  7, 2,  True), (30,  6, 2,  True),
           (31,  5, 2,  True), (32,  9, 2,  True), (33,  2, 2,  True),
           (34, 15, 2,  True), (35,  4, 2,  True), (36,  8, 1,  True),
           (37,  8, 1,  True), (38,  8, 1,  True), (39,  9, 1, False),
           (40,  5, 1, False), (41,  5, 1, False), (42,  4, 1, False),
      

In [3]:
# As the environment progresses througout time, the upcoming forecast adjusts:
env.timeout(10)
env.run()
env.state

rec.array([(11,  7, 2, False), (12,  3, 2, False), (13,  2, 3, False),
           (14,  3, 3, False), (15,  4, 3, False), (16,  5, 3, False),
           (17,  1, 2, False), (18,  2, 2, False), (19,  5, 2, False),
           (20,  8, 2, False), (21,  9, 2, False), (22,  8, 2, False),
           (23,  5, 3, False), (24,  4, 3, False), (25,  3, 3, False),
           (26,  2, 3,  True), (27,  1, 3,  True), (28,  8, 3,  True),
           (29,  7, 2,  True), (30,  6, 2,  True), (31,  5, 2,  True),
           (32,  9, 2,  True), (33,  2, 2,  True), (34, 15, 2,  True),
           (35,  4, 2,  True), (36,  8, 1,  True), (37,  8, 1,  True),
           (38,  8, 1,  True), (39,  9, 1, False), (40,  5, 1, False),
           (41,  5, 1, False), (42,  4, 1, False), (43,  3, 1, False),
           (44,  2, 1, False), (45,  8, 1, False), (46,  9, 2, False),
           (47,  7, 1, False), (48, 12, 1, False), (49,  4, 1, False),
           (50,  8, 1, False)],
          dtype=[('datetime', '<i8'), ('winds

In [11]:
# Reset env, create an agent and register it
env = Environment("Test Environment", state=data)
agent = Agent("Test Agent")
env.register(agent)

# A task that is submitted without a constraint will be processed immediately
agent.task("Run", duration=10, suspendable=True)
env.run()
env.now

10